<a href="https://colab.research.google.com/github/seelca/more-access/blob/dev/more_access_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#to do next:
#load points from API https://data.bus-data.dft.gov.uk/api/buslocation-openapi/#/SIRI-VM%20Data%20feed/get_api_v1_datafeed
#plot them into map

#load other datasets
#see if we can find in folium an option to switch data on and off (like layers)
#bus api key 	aa059cefd148311cdd0f5c4dd05a48ecd5fc868f
!pip install osmnx
!pip install folium
!pip install requests

# Data gathering


In [ ]:
#bus stops
#https://naptan.api.dft.gov.uk/swagger/index.html
import requests
from google.colab import files

# The URL from which to download the CSV file
url = 'https://naptan.api.dft.gov.uk/v1/access-nodes?dataFormat=csv&atcoAreaCodes=049'

# Make a GET request to download the CSV file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Define the filename you want to save the CSV as
    filename = "naptan.csv"

    # Write the content to the file in the Colab environment
    with open(filename, 'wb') as file:
        file.write(response.content)

    # Use the files.download function to download the file to your local machine
    files.download(filename)

    print(f"File downloaded and available for local download as {filename}")
else:
    print(f"Failed to retrieve the CSV file: {response.status_code}, Response: {response.text}")

# Read the CSV into a DataFrame
filename = "naptan.csv"
df = pd.read_csv(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

File downloaded and available for local download as naptan.csv


In [59]:
#nextbike
import requests

def fetch_bike_data(city_uid=None, lat=None, lng=None, limit=100, distance=2000):
    base_url = "https://maps.nextbike.net/maps/nextbike-live.json"
    params = {
        "city": city_uid,
        "lat": lat,
        "lng": lng,
        "limit": limit,
        "distance": distance
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Coordinates roughly central to Milton Keynes
lat = 52.0406224
lng = -0.7594171

# Fetch bike data using geographic coordinates
bike_data = fetch_bike_data(lat=lat, lng=lng)

if bike_data:
    print("ok")
else:
    print("No data available for the specified coordinates.")

ok


# Plot data

In [ ]:
#libraries
import pandas as pd
import folium
from folium.plugins import MarkerCluster

In [ ]:
# Define a generic PointOfInterest class
class PointOfInterest:
    def __init__(self, name, latitude, longitude, popup_info=None, marker_color='blue'):
        self.name = name
        self.latitude = latitude
        self.longitude = longitude
        self.popup_info = popup_info
        self.marker_color = marker_color

    def add_to_map(self, map_object):
        folium.Marker(
            location=[self.latitude, self.longitude],
            popup=self.popup_info if self.popup_info else self.name,
            icon=folium.Icon(color=self.marker_color)
        ).add_to(map_object)

In [58]:
#plot data
# Instantiate PointOfInterest objects for bus stops from the DataFrame
bus_stops = [
    PointOfInterest(
        name=row['CommonName'],
        latitude=row['Latitude'],
        longitude=row['Longitude'],
        popup_info=f"{row['CommonName']} ({row['Indicator']})",
        marker_color='blue'
    )
    for index, row in df.iterrows()
]

# Assuming bike_data is the JSON response you've got from the Nextbike API
bike_stations = []
for country in bike_data.get('countries', []):
    for city in country.get('cities', []):
        for place in city.get('places', []):
            if place.get('bike', False) is False:  # Ensures we're dealing with stations, not free-floating bikes
                bike_stations.append(
                    PointOfInterest(
                        name=place.get('name'),
                        latitude=place.get('lat'),
                        longitude=place.get('lng'),
                        popup_info=f"Bikes Available: {place.get('bikes')}",
                        marker_color='green'  # Custom marker color for bike stations
                    )
                )


# Create the base map
# Set the city name or location
place_name = "Milton Keynes, United Kingdom"
map = folium.Map(location=[52.04172, -0.75583], zoom_start=15)

# Create different FeatureGroups for each type of data
bus_stops_layer = folium.FeatureGroup(name='Bus Stops')
bike_stations_layer = folium.FeatureGroup(name='Bike Stations')

# Add bus stops to the map
for poi in bus_stops:
    poi.add_to_map(bus_stops_layer)

#add bikes
# Add bike stations to the bike stations layer
for station in bike_stations:
    station.add_to_map(bike_stations_layer)

# Add FeatureGroups to the map
bus_stops_layer.add_to(map)
bike_stations_layer.add_to(map)

# Add LayerControl to the map to allow layer switching
folium.LayerControl(collapsed=False).add_to(map)

# Display the map
map


In [ ]:
import folium
import pandas as pd

# Create the base map
map = folium.Map(location=[52.04172, -0.75583], zoom_start=15)

# Example data for parks and restaurants
parks_data = [
    {'name': 'Park A', 'latitude': 52.04572, 'longitude': -0.75583},
    {'name': 'Park B', 'latitude': 52.04072, 'longitude': -0.76083}
]

restaurants_data = [
    {'name': 'Restaurant A', 'latitude': 52.04672, 'longitude': -0.75083},
    {'name': 'Restaurant B', 'latitude': 52.04272, 'longitude': -0.75883}
]

# Create different FeatureGroups
bus_stops_layer = folium.FeatureGroup(name='Bus Stops')
parks_layer = folium.FeatureGroup(name='Parks')
restaurants_layer = folium.FeatureGroup(name='Restaurants')

# Add example parks and restaurants to the respective layers
for park in parks_data:
    folium.Marker(
        location=[park['latitude'], park['longitude']],
        popup=park['name']
    ).add_to(parks_layer)

for restaurant in restaurants_data:
    folium.Marker(
        location=[restaurant['latitude'], restaurant['longitude']],
        popup=restaurant['name']
    ).add_to(restaurants_layer)

# Add all FeatureGroups to the map
bus_stops_layer.add_to(map)
parks_layer.add_to(map)
restaurants_layer.add_to(map)

# Add a LayerControl to allow layer switching
folium.LayerControl(collapsed=False).add_to(map)

# Display the map
map


In [ ]:
import osmnx as ox
import folium

# Set the city name or location
place_name = "Milton Keynes, United Kingdom"
map = folium.Map(location=[52.04172, -0.75583], zoom_start=15)
map


In [ ]:
#add data (markers)
folium.Marker([52.039846393280015, -0.7505210635752713], popup="point plotted", icon=folium.Icon(color="greeen", icon="glyphicon glyphicon-plus")).add_to(map)

map

<ipython-input-22-0c6f4ea0e2c4>:2: UserWarning: color argument of Icon should be one of: {'blue', 'black', 'green', 'orange', 'pink', 'cadetblue', 'darkred', 'white', 'darkpurple', 'beige', 'gray', 'lightgreen', 'darkblue', 'purple', 'darkgreen', 'lightred', 'lightblue', 'lightgray', 'red'}.
  folium.Marker([52.039846393280015, -0.7505210635752713], popup="point plotted", icon=folium.Icon(color="greeen", icon="glyphicon glyphicon-plus")).add_to(map)


In [ ]:
folium.Circle(
    radius=1000,
    color="green",
    location=[52.039846393280015, -0.7505210635752713],
    fill=True).add_to(map)

map